In [1]:
import os
os.chdir('/nfs/homedirs/ayle/guided-research/SNIP-it/')

In [2]:
import torch
from torchvision import datasets, transforms
import foolbox as fb
from experiments.main import load_checkpoint
from models import GeneralModel
from models.statistics.Metrics import Metrics
from utils.config_utils import *
from utils.model_utils import *
from utils.system_utils import *
from utils.attacks_utils import get_attack
from torch.utils.data.dataset import Dataset
from copy import deepcopy
from utils.metrics import calculate_aupr, calculate_auroc
from utils.attacks_utils import construct_adversarial_examples
import matplotlib.pyplot as plt
from tqdm import tqdm
import torch.nn.functional as F
from torch.distributions import Categorical

In [3]:
arguments = dict({
'eval_freq': 1000,  # evaluate every n batches
    'save_freq': 1e6,  # save model every n epochs, besides before and after training
    'batch_size': 512,  # size of batches, for Imagenette 128
    'seed': 1234,  # random seed
    'max_training_minutes': 6120 , # one hour and a 45 minutes max, process killed after n minutes (after finish of epoch)
    'plot_weights_freq': 50, # plot pictures to tensorboard every n epochs
    'prune_freq': 1, # if pruning during training: how long to wait before starting
    'prune_delay': 0, # "if pruning during training: 't' from algorithm box, interval between pruning events, default=0
    'prune_to': 0,
    'epochs': 0,
    'rewind_to': 0, # rewind to this epoch if rewinding is done
    'snip_steps': 5, # 's' in algorithm box, number of pruning steps for 'rule of thumb', TODO
    'snip_iter': 1000,
    'pruning_rate': 0.0, # pruning rate passed to criterion at pruning event. however, most override this
    'growing_rate': 0.0000 , # grow back so much every epoch (for future criterions)
    'pruning_limit': 0.0,  # Prune until here, if structured in nodes, if unstructured in weights. most criterions use this instead of the pruning_rate
    'local_pruning': 0,
    'learning_rate': 2e-3,
    'grad_clip': 10,
    'grad_noise': 0 , # added gaussian noise to gradients
    'l2_reg': 5e-5 , # weight decay
    'l1_reg': 0 , # l1-norm regularisation
    'lp_reg': 0 , # lp regularisation with p < 1
    'l0_reg': 1.0 , # l0 reg lambda hyperparam
    'hoyer_reg': 0.001 , # hoyer reg lambda hyperparam
    'beta_ema': 0.999 , # l0 reg beta ema hyperparam

    'loss': 'CrossEntropy',
    'optimizer': 'ADAM',
    'model': 'ResNet18',  # ResNet not supported with structured
    'data_set': 'CIFAR10',
    'ood_data_set': 'SVHN',
    'ood_data_set_prune': 'SVHN',
    'prune_criterion': 'WeightImportance',  # options: SNIP, SNIPit, SNIPitDuring, UnstructuredRandom, GRASP, HoyerSquare, IMP, // SNAPit, StructuredRandom, GateDecorators, EfficientConvNets, GroupHoyerSquare
    'train_scheme': 'DefaultTrainer' , # default: DefaultTrainer
    'attack': 'FGSM',
    'epsilon': 6,
    'eval_ood_data_sets': ['SVHN', 'CIFAR100'],
    'eval_attacks': ['FGSM'],
    'eval_epsilons': [8, 48],

    'device': 'cuda',
    'results_dir': "tmp",

    'checkpoint_name': None,
    'checkpoint_model': None,

    'disable_cuda_benchmark': 1 , # speedup (disable) vs reproducibility (leave it)
    'eval': 0,
    'disable_autoconfig': 0 , # for the brave
    'preload_all_data': 0 , # load all data into ram memory for speedups
    'tuning': 0 , # splits trainset into train and validationset, omits test set

    'get_hooks': 0,
    'track_weights': 0 , # "keep statistics on the weights through training
    'disable_masking': 1 , # disable the ability to prune unstructured
    'enable_rewinding': 0, # enable the ability to rewind to previous weights
    'outer_layer_pruning': 1, # allow to prune outer layers (unstructured) or not (structured)
    'first_layer_dense': 0,
    'random_shuffle_labels': 0  ,# run with random-label experiment from zhang et al
    'l0': 0,  # run with l0 criterion, might overwrite some other arguments
    'hoyer_square': 0, # "run in unstructured DeephoyerSquare criterion, might overwrite some other arguments
    'group_hoyer_square': 0 ,# run in unstructured Group-DeephoyerSquare criterion, might overwrite some other arguments

    'disable_histograms': 0,
    'disable_saliency': 0,
    'disable_confusion': 0,
    'disable_weightplot': 0,
    'disable_netplot': 0,
    'skip_first_plot': 0,
    'disable_activations': 0,
    
#     'input_dim': [1, 28, 28],
#       'output_dim': 10,
#       'hidden_dim': [512],
#       'N': 60000,
    
    'input_dim': [3, 32, 32],
      'output_dim': 10,
      'hidden_dim': [512],
      'N': 60000
})

In [4]:
DATASET_PATH = '/nfs/students/ayle/guided-research/gitignored/data'

In [5]:
metrics = Metrics()
out = metrics.log_line
metrics._batch_size = arguments['batch_size']
metrics._eval_freq = arguments['eval_freq']
set_results_dir(arguments["results_dir"])

In [6]:
model: GeneralModel = find_right_model(
        NETWORKS_DIR, arguments['model'],
        device=arguments['device'],
        hidden_dim=arguments['hidden_dim'],
        input_dim=arguments['input_dim'],
        output_dim=arguments['output_dim'],
        is_maskable=arguments['disable_masking'],
        is_tracking_weights=arguments['track_weights'],
        is_rewindable=arguments['enable_rewinding'],
        is_growable=arguments['growing_rate'] > 0,
        outer_layer_pruning=arguments['outer_layer_pruning'],
        maintain_outer_mask_anyway=(
                                       not arguments['outer_layer_pruning']) and (
                                           "Structured" in arguments['prune_criterion']),
        l0=arguments['l0'],
        l0_reg=arguments['l0_reg'],
        N=arguments['N'],
        beta_ema=arguments['beta_ema'],
        l2_reg=arguments['l2_reg']
).to(arguments['device'])

10


In [7]:
load_checkpoint(arguments, model, out)

In [8]:
def load_checkpoint(path, model, out):
    with open(path, 'rb') as f:
        state = pickle.load(f)
    try:
        model.load_state_dict(state)
    except KeyError as e:
        print(list(state.keys()))
        raise e
    out(f"Loaded checkpoint {path}")

In [9]:
# path = '/nfs/students/ayle/guided-research/results/Conv6/2021-07-12_03.45.39_model=Conv6_dataset=CIFAR10_prune-criterion=EmptyCrit_pruning-limit=0.0_prune-freq=1_prune-delay=0_outer-layer-pruning=1_prune-to=10_rewind-to=0_train-scheme=DefaultTrainer_seed=1234/models/Conv6_finished.pickle'
# path = '/nfs/students/ayle/guided-research/results/Conv6/2021-07-12_04.48.17_model=Conv6_dataset=CIFAR10_prune-criterion=EmptyCrit_pruning-limit=0.0_prune-freq=1_prune-delay=0_outer-layer-pruning=1_prune-to=10_rewind-to=0_train-scheme=DefaultTrainer_seed=2345/models/Conv6_finished.pickle'
# path = '/nfs/students/ayle/guided-research/results/Conv6/2021-07-15_19.21.19_model=Conv6_dataset=CIFAR10_prune-criterion=EmptyCrit_pruning-limit=0.0_prune-freq=1_prune-delay=0_outer-layer-pruning=1_prune-to=10_rewind-to=0_train-scheme=DefaultTrainer_seed=3456/models/Conv6_finished.pickle'
# path = '/nfs/students/ayle/guided-research/results/Conv6/2021-07-15_19.25.40_model=Conv6_dataset=CIFAR10_prune-criterion=EmptyCrit_pruning-limit=0.0_prune-freq=1_prune-delay=0_outer-layer-pruning=1_prune-to=10_rewind-to=0_train-scheme=DefaultTrainer_seed=4567/models/Conv6_finished.pickle'
# path = '/nfs/students/ayle/guided-research/results/Conv6/2021-07-15_19.25.40_model=Conv6_dataset=CIFAR10_prune-criterion=EmptyCrit_pruning-limit=0.0_prune-freq=1_prune-delay=0_outer-layer-pruning=1_prune-to=10_rewind-to=0_train-scheme=DefaultTrainer_seed=4567/models/Conv6_finished.pickle'

# path = '/nfs/students/ayle/guided-research/results/LeNet5/2021-07-11_03.10.29_model=LeNet5_dataset=FASHION_prune-criterion=EmptyCrit_pruning-limit=0.0_prune-freq=1_prune-delay=0_outer-layer-pruning=1_prune-to=10_rewind-to=0_train-scheme=DefaultTrainer_seed=1234/models/LeNet5_finished.pickle'

# path = '/nfs/students/ayle/guided-research/results/ResNet18/2021-07-13_11.03.15_model=ResNet18_dataset=CIFAR10_prune-criterion=EmptyCrit_pruning-limit=0.0_prune-freq=1_prune-delay=0_outer-layer-pruning=1_prune-to=10_rewind-to=0_train-scheme=DefaultTrainer_seed=1234/models/ResNet18_finished.pickle'

# path = '/nfs/students/ayle/guided-research/results/ResNet18/2021-07-26_22.46.19_model=ResNet18_dataset=CIFAR10_prune-criterion=EmptyCrit_pruning-limit=0.0_prune-freq=1_prune-delay=0_outer-layer-pruning=1_prune-to=10_rewind-to=0_train-scheme=DefaultTrainer_seed=1234/models/ResNet18_finished.pickle'
# path2 = '/nfs/students/ayle/guided-research/results/ResNet18/2021-07-26_23.33.17_model=ResNet18_dataset=CIFAR10_prune-criterion=EmptyCrit_pruning-limit=0.0_prune-freq=1_prune-delay=0_outer-layer-pruning=1_prune-to=10_rewind-to=0_train-scheme=DefaultTrainer_seed=2345/models/ResNet18_finished.pickle'
# path = '/nfs/homedirs/ayle/guided-research/SNIP-it/gitignored/results/ResNet18/2021-07-26_23.35.18_model=ResNet18_dataset=CIFAR10_prune-criterion=EmptyCrit_pruning-limit=0.0_prune-freq=1_prune-delay=0_outer-layer-pruning=1_prune-to=10_rewind-to=0_train-scheme=DefaultTrainer_seed=3456/models/ResNet18_finished.pickle'
# path = '/nfs/homedirs/ayle/guided-research/SNIP-it/gitignored/results/ResNet18/2021-07-26_23.35.46_model=ResNet18_dataset=CIFAR10_prune-criterion=EmptyCrit_pruning-limit=0.0_prune-freq=1_prune-delay=0_outer-layer-pruning=1_prune-to=10_rewind-to=0_train-scheme=DefaultTrainer_seed=4567/models/ResNet18_finished.pickle'
# path = '/nfs/homedirs/ayle/guided-research/SNIP-it/gitignored/results/ResNet18/2021-07-26_23.36.23_model=ResNet18_dataset=CIFAR10_prune-criterion=EmptyCrit_pruning-limit=0.0_prune-freq=1_prune-delay=0_outer-layer-pruning=1_prune-to=10_rewind-to=0_train-scheme=DefaultTrainer_seed=5678/models/ResNet18_finished.pickle'

# path = '/nfs/homedirs/ayle/guided-research/SNIP-it/gitignored/results/VGG16/2021-08-22_11.02.10_model=VGG16_dataset=CIFAR10_prune-criterion=EmptyCrit_pruning-limit=0.0_prune-freq=1_prune-delay=0_outer-layer-pruning=1_prune-to=10_rewind-to=0_train-scheme=DefaultTrainer_seed=1234/models/VGG16_finished.pickle'

# path = '/nfs/homedirs/ayle/guided-research/SNIP-it/gitignored/results/ResNet18/2021-08-29_13.57.22_model=ResNet18_dataset=CIFAR10_prune-criterion=EmptyCrit_pruning-limit=0.0_prune-freq=1_prune-delay=0_outer-layer-pruning=1_prune-to=10_rewind-to=0_train-scheme=DefaultTrainer_seed=1234/models/ResNet18_finished.pickle'

# path = '/nfs/homedirs/ayle/guided-research/SNIP-it/gitignored/results/ResNet18/2021-08-29_18.19.04_model=ResNet18_dataset=CIFAR10_prune-criterion=EmptyCrit_pruning-limit=0.0_prune-freq=1_prune-delay=0_outer-layer-pruning=1_prune-to=10_rewind-to=0_train-scheme=DefaultTrainer_seed=1234/models/ResNet18_finished.pickle'

# path = '/nfs/homedirs/ayle/guided-research/SNIP-it/gitignored/results/ResNet18/2021-08-24_11.36.47_model=ResNet18_dataset=CIFAR10_prune-criterion=EarlyJohn_pruning-limit=0.94_prune-freq=1_prune-delay=0_outer-layer-pruning=1_prune-to=10_rewind-to=0_train-scheme=DefaultTrainer_seed=1234/models/ResNet18_finished.pickle'

path = '/nfs/students/ayle/guided-research/gitignored/results/tests/2021-09-21_10.19.16_model=ResNet18_dataset=CIFAR10_prune-criterion=EmptyCrit_pruning-limit=0.0_train-scheme=DefaultTrainer_seed=1234/models/ResNet18_finished.pickle'

load_checkpoint(path, model, out)


Loaded checkpoint /nfs/students/ayle/guided-research/gitignored/results/tests/2021-09-21_10.19.16_model=ResNet18_dataset=CIFAR10_prune-criterion=EmptyCrit_pruning-limit=0.0_train-scheme=DefaultTrainer_seed=1234/models/ResNet18_finished.pickle


In [10]:
device = arguments['device']

In [11]:
# load data
train_loader, test_loader = find_right_model(
    DATASETS, arguments['data_set'],
    arguments=arguments
)

# load OOD data
_, ood_loader = find_right_model(
    DATASETS, arguments['ood_data_set'],
    arguments=arguments
)

Files already downloaded and verified
Files already downloaded and verified
Using downloaded and verified file: /nfs/students/ayle/guided-research/gitignored/data/train_32x32.mat
Using downloaded and verified file: /nfs/students/ayle/guided-research/gitignored/data/test_32x32.mat


In [12]:
# load OOD data
ood_prune_loader, _ = find_right_model(
    DATASETS, arguments['ood_data_set_prune'],
    arguments=arguments
)

# get loss function
loss = find_right_model(
    LOSS_DIR, arguments['loss'],
    device=device,
    l1_reg=arguments['l1_reg'],
    lp_reg=arguments['lp_reg'],
    l0_reg=arguments['l0_reg'],
    hoyer_reg=arguments['hoyer_reg']
)

# get optimizer
optimizer = find_right_model(
    OPTIMS, arguments['optimizer'],
    params=model.parameters(),
    lr=arguments['learning_rate'],
    weight_decay=arguments['l2_reg'] if not arguments['l0'] else 0
)

Using downloaded and verified file: /nfs/students/ayle/guided-research/gitignored/data/train_32x32.mat
Using downloaded and verified file: /nfs/students/ayle/guided-research/gitignored/data/test_32x32.mat


In [13]:
backup_model = deepcopy(model)

In [14]:
run_name = f'_model={arguments["model"]}_dataset={arguments["data_set"]}_ood-dataset={arguments["ood_data_set"]}' + \
           f'_attack={arguments["attack"]}_epsilon={arguments["epsilon"]}_prune-criterion={arguments["prune_criterion"]}' + \
           f'_pruning-limit={arguments["pruning_limit"]}_prune-freq={arguments["prune_freq"]}_prune-delay={arguments["prune_delay"]}' + \
           f'_rewind-to={arguments["rewind_to"]}_train-scheme={arguments["train_scheme"]}_seed={arguments["seed"]}'


criterion = find_right_model(
        CRITERION_DIR, arguments['prune_criterion'],
        model=model,
        limit=arguments['pruning_limit'],
        start=0.5,
        orig_scores=True,
        steps=arguments['snip_steps'],
        device=arguments['device'],
        arguments=arguments
    )

# build trainer
trainer = find_right_model(
    TRAINERS_DIR, arguments['train_scheme'],
    model=model,
    loss=loss,
    optimizer=optimizer,
    device=device,
    arguments=arguments,
    train_loader=train_loader,
    test_loader=test_loader,
    ood_loader=ood_loader,
    ood_prune_loader=ood_prune_loader,
    metrics=metrics,
    criterion=criterion,
    run_name=run_name
)

trainer.train()

Made datestamp: 2021-09-22_10.50.19_model=ResNet18_dataset=CIFAR10_ood-dataset=SVHN_attack=FGSM_epsilon=6_prune-criterion=WeightImportance_pruning-limit=0.0_prune-freq=1_prune-delay=0_rewind-to=0_train-scheme=DefaultTrainer_seed=1234


  0%|          | 0/98 [00:00<?, ?it/s]

Started training


100%|██████████| 98/98 [00:20<00:00,  4.69it/s]


Saved results/tmp/2021-09-22_10.50.19_model=ResNet18_dataset=CIFAR10_ood-dataset=SVHN_attack=FGSM_epsilon=6_prune-criterion=WeightImportance_pruning-limit=0.0_prune-freq=1_prune-delay=0_rewind-to=0_train-scheme=DefaultTrainer_seed=1234/output/scores


In [15]:
orig_grads = criterion.grads_abs

In [16]:
orig_mean = criterion.scores_mean
orig_std = criterion.scores_std
layer_names = list(orig_grads.keys())

In [17]:
for name, val in orig_grads.items():
    orig_grads[name] = (val - orig_mean[name]) / (1e-8 + orig_std[name])

In [18]:
# backup_model = deepcopy(model)

In [19]:
arguments['batch_size'] = 1

# load data
train_loader, test_loader = find_right_model(
    DATASETS, arguments['data_set'],
    arguments=arguments
)

# load OOD data
_, ood_loader = find_right_model(
    DATASETS, arguments['ood_data_set'],
    arguments=arguments
)

Files already downloaded and verified
Files already downloaded and verified
Using downloaded and verified file: /nfs/students/ayle/guided-research/gitignored/data/train_32x32.mat
Using downloaded and verified file: /nfs/students/ayle/guided-research/gitignored/data/test_32x32.mat


In [20]:
norms = []
per_layer_norms = []
for i, (x, y) in enumerate(tqdm(test_loader)):
    if i == int(len(test_loader)*0.1): break
        
    train_x, train_y = next(iter(train_loader))
    
    model = deepcopy(backup_model)
    model.eval()
        
    batch_x = torch.cat((train_x, x))
    batch_y = torch.cat((train_y, y))
    out = model(batch_x.cuda())
    preds = out.argmax(dim=-1, keepdim=True).view_as(batch_y)
        
    # get criterion
    criterion = find_right_model(
        CRITERION_DIR, arguments['prune_criterion'],
        model=model,
        limit=arguments['pruning_limit'],
        start=0.5,
        steps=arguments['snip_steps'],
        device=arguments['device'],
        arguments=arguments
    )
    
    criterion.prune(arguments['pruning_limit'],
                        train_loader=[(batch_x, preds)],
                      ood_loader=None,
                      local=arguments['local_pruning'],
                      manager=None)
    
    layer_norms = []
    for j, (grad1, grad2) in enumerate(zip(orig_grads.values(), criterion.grads_abs.values())):
        grad3 = (grad2 - orig_mean[layer_names[j]]) /  (1e-8 + orig_std[layer_names[j]])
        layer_norms.append(torch.norm(grad1 - grad3, p=2).cpu().detach().numpy())

    train_x2, train_y2 = next(iter(train_loader))
    
    batch_x = torch.cat((train_x, train_x2))
    batch_y = torch.cat((train_y, train_y2))
    out = model(batch_x.cuda())
    preds = out.argmax(dim=-1, keepdim=True).view_as(batch_y)
        
    # get criterion
    criterion = find_right_model(
        CRITERION_DIR, arguments['prune_criterion'],
        model=model,
        limit=arguments['pruning_limit'],
        start=0.5,
        steps=arguments['snip_steps'],
        device=arguments['device'],
        arguments=arguments
    )
    
    criterion.prune(arguments['pruning_limit'],
                        train_loader=[(batch_x, preds)],
                      ood_loader=None,
                      local=arguments['local_pruning'],
                      manager=None)
    
    layer_norms_train = []
    for j, (grad1, grad2) in enumerate(zip(orig_grads.values(), criterion.grads_abs.values())):
        grad3 = (grad2 - orig_mean[layer_names[j]]) /  (1e-8 + orig_std[layer_names[j]])
        layer_norms_train.append(torch.norm(grad1 - grad3, p=2).cpu().detach().numpy())
    
    norms.append(np.mean(layer_norms) - np.mean(layer_norms_train))

100%|██████████| 1/1 [00:00<00:00, 66.84it/s]

100%|██████████| 1/1 [00:00<00:00, 118.46it/s]

100%|██████████| 1/1 [00:00<00:00, 124.79it/s]

100%|██████████| 1/1 [00:00<00:00, 125.90it/s]

100%|██████████| 1/1 [00:00<00:00, 125.70it/s]

100%|██████████| 1/1 [00:00<00:00, 122.78it/s]

100%|██████████| 1/1 [00:00<00:00, 126.43it/s]

100%|██████████| 1/1 [00:00<00:00, 126.35it/s]

100%|██████████| 1/1 [00:00<00:00, 124.43it/s]

100%|██████████| 1/1 [00:00<00:00, 125.23it/s]

100%|██████████| 1/1 [00:00<00:00, 124.74it/s]

100%|██████████| 1/1 [00:00<00:00, 126.50it/s]

100%|██████████| 1/1 [00:00<00:00, 123.34it/s]

100%|██████████| 1/1 [00:00<00:00, 123.49it/s]

100%|██████████| 1/1 [00:00<00:00, 125.34it/s]

100%|██████████| 1/1 [00:00<00:00, 123.40it/s]

100%|██████████| 1/1 [00:00<00:00, 123.80it/s]

100%|██████████| 1/1 [00:00<00:00, 124.52it/s]

100%|██████████| 1/1 [00:00<00:00, 123.29it/s]

100%|██████████| 1/1 [00:00<00:00, 125.72it/s]

100%|██████████| 1/1 [00:00<00:00, 120.95

100%|██████████| 1/1 [00:00<00:00, 125.47it/s]

100%|██████████| 1/1 [00:00<00:00, 125.70it/s]

100%|██████████| 1/1 [00:00<00:00, 121.42it/s]

100%|██████████| 1/1 [00:00<00:00, 125.39it/s]

100%|██████████| 1/1 [00:00<00:00, 126.36it/s]

100%|██████████| 1/1 [00:00<00:00, 125.15it/s]

100%|██████████| 1/1 [00:00<00:00, 124.79it/s]

100%|██████████| 1/1 [00:00<00:00, 123.60it/s]

100%|██████████| 1/1 [00:00<00:00, 123.57it/s]

100%|██████████| 1/1 [00:00<00:00, 125.30it/s]

100%|██████████| 1/1 [00:00<00:00, 122.94it/s]

100%|██████████| 1/1 [00:00<00:00, 125.85it/s]

100%|██████████| 1/1 [00:00<00:00, 125.53it/s]

100%|██████████| 1/1 [00:00<00:00, 124.86it/s]

100%|██████████| 1/1 [00:00<00:00, 123.58it/s]

100%|██████████| 1/1 [00:00<00:00, 124.71it/s]

100%|██████████| 1/1 [00:00<00:00, 123.58it/s]

100%|██████████| 1/1 [00:00<00:00, 126.30it/s]

100%|██████████| 1/1 [00:00<00:00, 125.12it/s]

100%|██████████| 1/1 [00:00<00:00, 125.65it/s]

100%|██████████| 1/1 [00:00<00:00, 124.0

100%|██████████| 1/1 [00:00<00:00, 125.71it/s]

100%|██████████| 1/1 [00:00<00:00, 123.26it/s]

100%|██████████| 1/1 [00:00<00:00, 125.99it/s]

100%|██████████| 1/1 [00:00<00:00, 123.21it/s]

100%|██████████| 1/1 [00:00<00:00, 125.59it/s]

100%|██████████| 1/1 [00:00<00:00, 126.18it/s]

100%|██████████| 1/1 [00:00<00:00, 124.12it/s]

100%|██████████| 1/1 [00:00<00:00, 126.23it/s]

100%|██████████| 1/1 [00:00<00:00, 126.48it/s]

100%|██████████| 1/1 [00:00<00:00, 124.76it/s]

100%|██████████| 1/1 [00:00<00:00, 124.33it/s]

100%|██████████| 1/1 [00:00<00:00, 126.33it/s]

100%|██████████| 1/1 [00:00<00:00, 123.08it/s]

100%|██████████| 1/1 [00:00<00:00, 125.84it/s]

100%|██████████| 1/1 [00:00<00:00, 126.24it/s]

100%|██████████| 1/1 [00:00<00:00, 124.65it/s]

100%|██████████| 1/1 [00:00<00:00, 121.23it/s]

100%|██████████| 1/1 [00:00<00:00, 125.74it/s]

100%|██████████| 1/1 [00:00<00:00, 124.47it/s]

100%|██████████| 1/1 [00:00<00:00, 125.82it/s]

100%|██████████| 1/1 [00:00<00:00, 125.5

100%|██████████| 1/1 [00:00<00:00, 125.35it/s]

100%|██████████| 1/1 [00:00<00:00, 125.05it/s]

100%|██████████| 1/1 [00:00<00:00, 124.62it/s]

100%|██████████| 1/1 [00:00<00:00, 123.93it/s]

100%|██████████| 1/1 [00:00<00:00, 123.18it/s]

100%|██████████| 1/1 [00:00<00:00, 123.78it/s]

100%|██████████| 1/1 [00:00<00:00, 123.73it/s]

100%|██████████| 1/1 [00:00<00:00, 124.46it/s]

100%|██████████| 1/1 [00:00<00:00, 124.33it/s]

100%|██████████| 1/1 [00:00<00:00, 125.60it/s]

100%|██████████| 1/1 [00:00<00:00, 122.18it/s]

100%|██████████| 1/1 [00:00<00:00, 123.57it/s]

100%|██████████| 1/1 [00:00<00:00, 122.38it/s]

100%|██████████| 1/1 [00:00<00:00, 126.17it/s]

100%|██████████| 1/1 [00:00<00:00, 123.57it/s]

100%|██████████| 1/1 [00:00<00:00, 124.45it/s]

100%|██████████| 1/1 [00:00<00:00, 124.77it/s]

100%|██████████| 1/1 [00:00<00:00, 123.39it/s]

100%|██████████| 1/1 [00:00<00:00, 124.41it/s]

100%|██████████| 1/1 [00:00<00:00, 124.33it/s]

100%|██████████| 1/1 [00:00<00:00, 125.6

100%|██████████| 1/1 [00:00<00:00, 125.46it/s]

100%|██████████| 1/1 [00:00<00:00, 125.12it/s]

100%|██████████| 1/1 [00:00<00:00, 124.60it/s]

100%|██████████| 1/1 [00:00<00:00, 121.04it/s]

100%|██████████| 1/1 [00:00<00:00, 123.24it/s]

100%|██████████| 1/1 [00:00<00:00, 123.67it/s]

100%|██████████| 1/1 [00:00<00:00, 122.25it/s]

100%|██████████| 1/1 [00:00<00:00, 122.88it/s]

100%|██████████| 1/1 [00:00<00:00, 125.11it/s]

100%|██████████| 1/1 [00:00<00:00, 125.90it/s]

100%|██████████| 1/1 [00:00<00:00, 125.06it/s]

100%|██████████| 1/1 [00:00<00:00, 120.91it/s]

100%|██████████| 1/1 [00:00<00:00, 123.15it/s]

100%|██████████| 1/1 [00:00<00:00, 125.34it/s]

100%|██████████| 1/1 [00:00<00:00, 124.85it/s]

100%|██████████| 1/1 [00:00<00:00, 126.08it/s]

100%|██████████| 1/1 [00:00<00:00, 124.08it/s]

100%|██████████| 1/1 [00:00<00:00, 123.99it/s]

100%|██████████| 1/1 [00:00<00:00, 124.34it/s]

100%|██████████| 1/1 [00:00<00:00, 123.85it/s]

100%|██████████| 1/1 [00:00<00:00, 123.8

...


100%|██████████| 1/1 [00:00<00:00, 125.52it/s]

100%|██████████| 1/1 [00:00<00:00, 123.07it/s]

100%|██████████| 1/1 [00:00<00:00, 121.68it/s]

100%|██████████| 1/1 [00:00<00:00, 124.06it/s]

100%|██████████| 1/1 [00:00<00:00, 125.54it/s]

100%|██████████| 1/1 [00:00<00:00, 126.34it/s]

100%|██████████| 1/1 [00:00<00:00, 125.23it/s]

100%|██████████| 1/1 [00:00<00:00, 125.31it/s]

100%|██████████| 1/1 [00:00<00:00, 124.04it/s]

100%|██████████| 1/1 [00:00<00:00, 124.43it/s]

100%|██████████| 1/1 [00:00<00:00, 126.06it/s]

100%|██████████| 1/1 [00:00<00:00, 125.67it/s]

100%|██████████| 1/1 [00:00<00:00, 124.56it/s]

100%|██████████| 1/1 [00:00<00:00, 125.81it/s]

100%|██████████| 1/1 [00:00<00:00, 125.63it/s]

100%|██████████| 1/1 [00:00<00:00, 124.44it/s]

100%|██████████| 1/1 [00:00<00:00, 120.26it/s]

100%|██████████| 1/1 [00:00<00:00, 122.95it/s]

100%|██████████| 1/1 [00:00<00:00, 124.08it/s]

100%|██████████| 1/1 [00:00<00:00, 122.82it/s]

100%|██████████| 1/1 [00:00<00:00, 123.

100%|██████████| 1/1 [00:00<00:00, 122.83it/s]

100%|██████████| 1/1 [00:00<00:00, 122.84it/s]

100%|██████████| 1/1 [00:00<00:00, 125.02it/s]

100%|██████████| 1/1 [00:00<00:00, 122.23it/s]

100%|██████████| 1/1 [00:00<00:00, 125.01it/s]

100%|██████████| 1/1 [00:00<00:00, 124.90it/s]

100%|██████████| 1/1 [00:00<00:00, 125.27it/s]

100%|██████████| 1/1 [00:00<00:00, 125.05it/s]

100%|██████████| 1/1 [00:00<00:00, 123.53it/s]

100%|██████████| 1/1 [00:00<00:00, 123.81it/s]

100%|██████████| 1/1 [00:00<00:00, 121.75it/s]

100%|██████████| 1/1 [00:00<00:00, 119.59it/s]

100%|██████████| 1/1 [00:00<00:00, 124.60it/s]

100%|██████████| 1/1 [00:00<00:00, 117.30it/s]

100%|██████████| 1/1 [00:00<00:00, 124.17it/s]

100%|██████████| 1/1 [00:00<00:00, 123.13it/s]

100%|██████████| 1/1 [00:00<00:00, 124.81it/s]

100%|██████████| 1/1 [00:00<00:00, 126.38it/s]

100%|██████████| 1/1 [00:00<00:00, 125.65it/s]

100%|██████████| 1/1 [00:00<00:00, 125.21it/s]

100%|██████████| 1/1 [00:00<00:00, 124.1

100%|██████████| 1/1 [00:00<00:00, 122.10it/s]

100%|██████████| 1/1 [00:00<00:00, 124.88it/s]

100%|██████████| 1/1 [00:00<00:00, 125.89it/s]

100%|██████████| 1/1 [00:00<00:00, 123.26it/s]

100%|██████████| 1/1 [00:00<00:00, 121.91it/s]

100%|██████████| 1/1 [00:00<00:00, 121.12it/s]

100%|██████████| 1/1 [00:00<00:00, 122.39it/s]

100%|██████████| 1/1 [00:00<00:00, 124.80it/s]

100%|██████████| 1/1 [00:00<00:00, 122.58it/s]

100%|██████████| 1/1 [00:00<00:00, 122.63it/s]

100%|██████████| 1/1 [00:00<00:00, 122.41it/s]

100%|██████████| 1/1 [00:00<00:00, 125.39it/s]

100%|██████████| 1/1 [00:00<00:00, 122.10it/s]

100%|██████████| 1/1 [00:00<00:00, 124.59it/s]

100%|██████████| 1/1 [00:00<00:00, 119.25it/s]

100%|██████████| 1/1 [00:00<00:00, 125.53it/s]

100%|██████████| 1/1 [00:00<00:00, 125.74it/s]

100%|██████████| 1/1 [00:00<00:00, 124.82it/s]

100%|██████████| 1/1 [00:00<00:00, 124.56it/s]

100%|██████████| 1/1 [00:00<00:00, 126.09it/s]

100%|██████████| 1/1 [00:00<00:00, 124.6

100%|██████████| 1/1 [00:00<00:00, 125.69it/s]

100%|██████████| 1/1 [00:00<00:00, 126.40it/s]

100%|██████████| 1/1 [00:00<00:00, 125.80it/s]

100%|██████████| 1/1 [00:00<00:00, 125.57it/s]

100%|██████████| 1/1 [00:00<00:00, 117.56it/s]

100%|██████████| 1/1 [00:00<00:00, 120.92it/s]

100%|██████████| 1/1 [00:00<00:00, 121.61it/s]

100%|██████████| 1/1 [00:00<00:00, 121.82it/s]

100%|██████████| 1/1 [00:00<00:00, 125.30it/s]

100%|██████████| 1/1 [00:00<00:00, 122.15it/s]

100%|██████████| 1/1 [00:00<00:00, 123.04it/s]

100%|██████████| 1/1 [00:00<00:00, 125.26it/s]

100%|██████████| 1/1 [00:00<00:00, 124.60it/s]

100%|██████████| 1/1 [00:00<00:00, 125.29it/s]

100%|██████████| 1/1 [00:00<00:00, 125.92it/s]

100%|██████████| 1/1 [00:00<00:00, 123.41it/s]

100%|██████████| 1/1 [00:00<00:00, 126.16it/s]

100%|██████████| 1/1 [00:00<00:00, 123.49it/s]

100%|██████████| 1/1 [00:00<00:00, 124.70it/s]

100%|██████████| 1/1 [00:00<00:00, 125.77it/s]

100%|██████████| 1/1 [00:00<00:00, 125.2

100%|██████████| 1/1 [00:00<00:00, 120.66it/s]

100%|██████████| 1/1 [00:00<00:00, 126.22it/s]

100%|██████████| 1/1 [00:00<00:00, 126.72it/s]

100%|██████████| 1/1 [00:00<00:00, 124.38it/s]

100%|██████████| 1/1 [00:00<00:00, 125.46it/s]

100%|██████████| 1/1 [00:00<00:00, 124.92it/s]

100%|██████████| 1/1 [00:00<00:00, 122.25it/s]

100%|██████████| 1/1 [00:00<00:00, 121.53it/s]

100%|██████████| 1/1 [00:00<00:00, 120.26it/s]

100%|██████████| 1/1 [00:00<00:00, 122.73it/s]

100%|██████████| 1/1 [00:00<00:00, 119.04it/s]

100%|██████████| 1/1 [00:00<00:00, 125.07it/s]

100%|██████████| 1/1 [00:00<00:00, 126.22it/s]

100%|██████████| 1/1 [00:00<00:00, 126.61it/s]

100%|██████████| 1/1 [00:00<00:00, 124.76it/s]

100%|██████████| 1/1 [00:00<00:00, 124.71it/s]

100%|██████████| 1/1 [00:00<00:00, 117.45it/s]

100%|██████████| 1/1 [00:00<00:00, 122.17it/s]

 10%|█         | 1000/10000 [20:08<3:03:38,  1.22s/it]Traceback (most recent call last):
  File "/nfs/homedirs/ayle/miniconda3/envs/gr/l

In [21]:
np.mean(norms)

-32948.875

In [ ]:
# Load CIFAR100 OOD data
_, ood_loader = find_right_model(
    DATASETS, "CIFAR100",
    arguments=arguments
)

In [22]:
# OOD 

ood_norms = []
for i, (x, y) in enumerate(tqdm(ood_loader)):
    if i == int(len(test_loader)*0.1): break
        
    train_x, train_y = next(iter(train_loader))
    
    model = deepcopy(backup_model)
    model.eval()
        
    batch_x = torch.cat((train_x, x))
    batch_y = torch.cat((train_y, y))
    out = model(batch_x.cuda())
    preds = out.argmax(dim=-1, keepdim=True).view_as(batch_y)
        
    # get criterion
    criterion = find_right_model(
        CRITERION_DIR, arguments['prune_criterion'],
        model=model,
        limit=arguments['pruning_limit'],
        start=0.5,
        steps=arguments['snip_steps'],
        device=arguments['device'],
        arguments=arguments
    )
    
    criterion.prune(arguments['pruning_limit'],
                        train_loader=[(batch_x, preds)],
                      ood_loader=None,
                      local=arguments['local_pruning'],
                      manager=None)
    
    layer_norms = []
    for j, (grad1, grad2) in enumerate(zip(orig_grads.values(), criterion.grads_abs.values())):
        grad3 = (grad2 - orig_mean[layer_names[j]]) /  (1e-8 + orig_std[layer_names[j]])
        layer_norms.append(torch.norm(grad1 - grad3, p=2).cpu().detach().numpy())

    train_x2, train_y2 = next(iter(train_loader))
    
    batch_x = torch.cat((train_x, train_x2))
    batch_y = torch.cat((train_y, train_y2))
    out = model(batch_x.cuda())
    preds = out.argmax(dim=-1, keepdim=True).view_as(batch_y)
        
    # get criterion
    criterion = find_right_model(
        CRITERION_DIR, arguments['prune_criterion'],
        model=model,
        limit=arguments['pruning_limit'],
        start=0.5,
        steps=arguments['snip_steps'],
        device=arguments['device'],
        arguments=arguments
    )
    
    criterion.prune(arguments['pruning_limit'],
                        train_loader=[(batch_x, preds)],
                      ood_loader=None,
                      local=arguments['local_pruning'],
                      manager=None)
    
    layer_norms_train = []
    for j, (grad1, grad2) in enumerate(zip(orig_grads.values(), criterion.grads_abs.values())):
        grad3 = (grad2 - orig_mean[layer_names[j]]) /  (1e-8 + orig_std[layer_names[j]])
        layer_norms_train.append(torch.norm(grad1 - grad3, p=2).cpu().detach().numpy())
    
    ood_norms.append(np.mean(layer_norms) - np.mean(layer_norms_train))

100%|██████████| 1/1 [00:00<00:00, 117.61it/s]

100%|██████████| 1/1 [00:00<00:00, 115.35it/s]

100%|██████████| 1/1 [00:00<00:00, 125.99it/s]

100%|██████████| 1/1 [00:00<00:00, 121.39it/s]

100%|██████████| 1/1 [00:00<00:00, 121.40it/s]

100%|██████████| 1/1 [00:00<00:00, 125.10it/s]

100%|██████████| 1/1 [00:00<00:00, 123.61it/s]

100%|██████████| 1/1 [00:00<00:00, 123.78it/s]

100%|██████████| 1/1 [00:00<00:00, 125.22it/s]

100%|██████████| 1/1 [00:00<00:00, 123.69it/s]

100%|██████████| 1/1 [00:00<00:00, 124.39it/s]

100%|██████████| 1/1 [00:00<00:00, 123.02it/s]

100%|██████████| 1/1 [00:00<00:00, 125.08it/s]

100%|██████████| 1/1 [00:00<00:00, 125.82it/s]

100%|██████████| 1/1 [00:00<00:00, 124.96it/s]

100%|██████████| 1/1 [00:00<00:00, 123.75it/s]

100%|██████████| 1/1 [00:00<00:00, 118.69it/s]

100%|██████████| 1/1 [00:00<00:00, 120.64it/s]

100%|██████████| 1/1 [00:00<00:00, 119.74it/s]

100%|██████████| 1/1 [00:00<00:00, 118.10it/s]

100%|██████████| 1/1 [00:00<00:00, 119.9

100%|██████████| 1/1 [00:00<00:00, 123.79it/s]

100%|██████████| 1/1 [00:00<00:00, 121.46it/s]

100%|██████████| 1/1 [00:00<00:00, 122.87it/s]

100%|██████████| 1/1 [00:00<00:00, 123.03it/s]

100%|██████████| 1/1 [00:00<00:00, 125.41it/s]

100%|██████████| 1/1 [00:00<00:00, 126.02it/s]

100%|██████████| 1/1 [00:00<00:00, 124.29it/s]

100%|██████████| 1/1 [00:00<00:00, 116.04it/s]

100%|██████████| 1/1 [00:00<00:00, 118.84it/s]

100%|██████████| 1/1 [00:00<00:00, 123.70it/s]

100%|██████████| 1/1 [00:00<00:00, 123.74it/s]

100%|██████████| 1/1 [00:00<00:00, 124.92it/s]

100%|██████████| 1/1 [00:00<00:00, 123.07it/s]

100%|██████████| 1/1 [00:00<00:00, 124.67it/s]

100%|██████████| 1/1 [00:00<00:00, 123.59it/s]

100%|██████████| 1/1 [00:00<00:00, 125.15it/s]

100%|██████████| 1/1 [00:00<00:00, 122.34it/s]

100%|██████████| 1/1 [00:00<00:00, 123.81it/s]

100%|██████████| 1/1 [00:00<00:00, 123.68it/s]

100%|██████████| 1/1 [00:00<00:00, 126.13it/s]

100%|██████████| 1/1 [00:00<00:00, 124.8

100%|██████████| 1/1 [00:00<00:00, 113.77it/s]

100%|██████████| 1/1 [00:00<00:00, 126.67it/s]

100%|██████████| 1/1 [00:00<00:00, 123.87it/s]

100%|██████████| 1/1 [00:00<00:00, 115.12it/s]

100%|██████████| 1/1 [00:00<00:00, 122.87it/s]

100%|██████████| 1/1 [00:00<00:00, 125.47it/s]

100%|██████████| 1/1 [00:00<00:00, 125.20it/s]

100%|██████████| 1/1 [00:00<00:00, 125.62it/s]

100%|██████████| 1/1 [00:00<00:00, 122.73it/s]

100%|██████████| 1/1 [00:00<00:00, 125.16it/s]

100%|██████████| 1/1 [00:00<00:00, 123.31it/s]

100%|██████████| 1/1 [00:00<00:00, 125.52it/s]

100%|██████████| 1/1 [00:00<00:00, 119.33it/s]

100%|██████████| 1/1 [00:00<00:00, 121.32it/s]

100%|██████████| 1/1 [00:00<00:00, 117.37it/s]

100%|██████████| 1/1 [00:00<00:00, 120.71it/s]

100%|██████████| 1/1 [00:00<00:00, 123.46it/s]

100%|██████████| 1/1 [00:00<00:00, 126.41it/s]

100%|██████████| 1/1 [00:00<00:00, 124.34it/s]

100%|██████████| 1/1 [00:00<00:00, 125.53it/s]

100%|██████████| 1/1 [00:00<00:00, 126.6

100%|██████████| 1/1 [00:00<00:00, 123.58it/s]

100%|██████████| 1/1 [00:00<00:00, 125.06it/s]

100%|██████████| 1/1 [00:00<00:00, 118.06it/s]

100%|██████████| 1/1 [00:00<00:00, 124.57it/s]

100%|██████████| 1/1 [00:00<00:00, 122.66it/s]

100%|██████████| 1/1 [00:00<00:00, 117.52it/s]

100%|██████████| 1/1 [00:00<00:00, 123.29it/s]

100%|██████████| 1/1 [00:00<00:00, 124.42it/s]

100%|██████████| 1/1 [00:00<00:00, 122.43it/s]

100%|██████████| 1/1 [00:00<00:00, 119.87it/s]

100%|██████████| 1/1 [00:00<00:00, 125.36it/s]

100%|██████████| 1/1 [00:00<00:00, 122.85it/s]

100%|██████████| 1/1 [00:00<00:00, 120.60it/s]

100%|██████████| 1/1 [00:00<00:00, 125.46it/s]

100%|██████████| 1/1 [00:00<00:00, 120.66it/s]

100%|██████████| 1/1 [00:00<00:00, 125.54it/s]

100%|██████████| 1/1 [00:00<00:00, 124.70it/s]

100%|██████████| 1/1 [00:00<00:00, 123.63it/s]

100%|██████████| 1/1 [00:00<00:00, 122.60it/s]

100%|██████████| 1/1 [00:00<00:00, 121.58it/s]

100%|██████████| 1/1 [00:00<00:00, 123.8

100%|██████████| 1/1 [00:00<00:00, 125.38it/s]

100%|██████████| 1/1 [00:00<00:00, 121.37it/s]

100%|██████████| 1/1 [00:00<00:00, 124.63it/s]

100%|██████████| 1/1 [00:00<00:00, 125.48it/s]

100%|██████████| 1/1 [00:00<00:00, 122.08it/s]

100%|██████████| 1/1 [00:00<00:00, 121.00it/s]

100%|██████████| 1/1 [00:00<00:00, 123.91it/s]

100%|██████████| 1/1 [00:00<00:00, 124.97it/s]

100%|██████████| 1/1 [00:00<00:00, 124.95it/s]

100%|██████████| 1/1 [00:00<00:00, 125.37it/s]

100%|██████████| 1/1 [00:00<00:00, 125.50it/s]

100%|██████████| 1/1 [00:00<00:00, 124.28it/s]

100%|██████████| 1/1 [00:00<00:00, 125.80it/s]

100%|██████████| 1/1 [00:00<00:00, 124.80it/s]

100%|██████████| 1/1 [00:00<00:00, 123.31it/s]

100%|██████████| 1/1 [00:00<00:00, 123.67it/s]

100%|██████████| 1/1 [00:00<00:00, 124.26it/s]

100%|██████████| 1/1 [00:00<00:00, 124.08it/s]

100%|██████████| 1/1 [00:00<00:00, 121.31it/s]

100%|██████████| 1/1 [00:00<00:00, 124.86it/s]

100%|██████████| 1/1 [00:00<00:00, 121.6

...

100%|██████████| 1/1 [00:00<00:00, 124.59it/s]

100%|██████████| 1/1 [00:00<00:00, 124.24it/s]

100%|██████████| 1/1 [00:00<00:00, 124.54it/s]

100%|██████████| 1/1 [00:00<00:00, 124.52it/s]

100%|██████████| 1/1 [00:00<00:00, 122.90it/s]

100%|██████████| 1/1 [00:00<00:00, 124.66it/s]

100%|██████████| 1/1 [00:00<00:00, 123.78it/s]

100%|██████████| 1/1 [00:00<00:00, 124.35it/s]

100%|██████████| 1/1 [00:00<00:00, 124.91it/s]

100%|██████████| 1/1 [00:00<00:00, 123.43it/s]

100%|██████████| 1/1 [00:00<00:00, 125.63it/s]

100%|██████████| 1/1 [00:00<00:00, 123.70it/s]

100%|██████████| 1/1 [00:00<00:00, 125.06it/s]

100%|██████████| 1/1 [00:00<00:00, 123.75it/s]

100%|██████████| 1/1 [00:00<00:00, 124.35it/s]

100%|██████████| 1/1 [00:00<00:00, 123.42it/s]

100%|██████████| 1/1 [00:00<00:00, 124.51it/s]

100%|██████████| 1/1 [00:00<00:00, 125.02it/s]

100%|██████████| 1/1 [00:00<00:00, 126.06it/s]

100%|██████████| 1/1 [00:00<00:00, 122.32it/s]

100%|██████████| 1/1 [00:00<00:00, 124.8

100%|██████████| 1/1 [00:00<00:00, 122.98it/s]

100%|██████████| 1/1 [00:00<00:00, 125.30it/s]

100%|██████████| 1/1 [00:00<00:00, 124.46it/s]

100%|██████████| 1/1 [00:00<00:00, 122.03it/s]

100%|██████████| 1/1 [00:00<00:00, 124.46it/s]

100%|██████████| 1/1 [00:00<00:00, 119.48it/s]

100%|██████████| 1/1 [00:00<00:00, 122.79it/s]

100%|██████████| 1/1 [00:00<00:00, 120.66it/s]

100%|██████████| 1/1 [00:00<00:00, 122.34it/s]

100%|██████████| 1/1 [00:00<00:00, 123.84it/s]

100%|██████████| 1/1 [00:00<00:00, 125.13it/s]

100%|██████████| 1/1 [00:00<00:00, 124.44it/s]

100%|██████████| 1/1 [00:00<00:00, 124.16it/s]

100%|██████████| 1/1 [00:00<00:00, 123.21it/s]

100%|██████████| 1/1 [00:00<00:00, 126.70it/s]

100%|██████████| 1/1 [00:00<00:00, 124.66it/s]

100%|██████████| 1/1 [00:00<00:00, 125.33it/s]

100%|██████████| 1/1 [00:00<00:00, 125.62it/s]

100%|██████████| 1/1 [00:00<00:00, 124.90it/s]

100%|██████████| 1/1 [00:00<00:00, 123.33it/s]

100%|██████████| 1/1 [00:00<00:00, 124.0


100%|██████████| 1/1 [00:00<00:00, 125.85it/s]

100%|██████████| 1/1 [00:00<00:00, 125.22it/s]

100%|██████████| 1/1 [00:00<00:00, 124.39it/s]

100%|██████████| 1/1 [00:00<00:00, 123.00it/s]

100%|██████████| 1/1 [00:00<00:00, 123.10it/s]

100%|██████████| 1/1 [00:00<00:00, 125.09it/s]

100%|██████████| 1/1 [00:00<00:00, 125.05it/s]

100%|██████████| 1/1 [00:00<00:00, 124.19it/s]

100%|██████████| 1/1 [00:00<00:00, 125.33it/s]

100%|██████████| 1/1 [00:00<00:00, 124.98it/s]

100%|██████████| 1/1 [00:00<00:00, 123.05it/s]

100%|██████████| 1/1 [00:00<00:00, 123.70it/s]

100%|██████████| 1/1 [00:00<00:00, 123.47it/s]

100%|██████████| 1/1 [00:00<00:00, 123.77it/s]

100%|██████████| 1/1 [00:00<00:00, 118.51it/s]

100%|██████████| 1/1 [00:00<00:00, 122.58it/s]

100%|██████████| 1/1 [00:00<00:00, 124.58it/s]

100%|██████████| 1/1 [00:00<00:00, 124.50it/s]

100%|██████████| 1/1 [00:00<00:00, 124.10it/s]

100%|██████████| 1/1 [00:00<00:00, 126.30it/s]

100%|██████████| 1/1 [00:00<00:00, 121.

100%|██████████| 1/1 [00:00<00:00, 125.97it/s]

100%|██████████| 1/1 [00:00<00:00, 124.23it/s]

100%|██████████| 1/1 [00:00<00:00, 125.08it/s]

100%|██████████| 1/1 [00:00<00:00, 123.46it/s]

100%|██████████| 1/1 [00:00<00:00, 123.95it/s]

100%|██████████| 1/1 [00:00<00:00, 124.77it/s]

100%|██████████| 1/1 [00:00<00:00, 122.33it/s]

100%|██████████| 1/1 [00:00<00:00, 125.18it/s]

100%|██████████| 1/1 [00:00<00:00, 123.69it/s]

100%|██████████| 1/1 [00:00<00:00, 120.28it/s]

100%|██████████| 1/1 [00:00<00:00, 123.55it/s]

100%|██████████| 1/1 [00:00<00:00, 125.49it/s]

100%|██████████| 1/1 [00:00<00:00, 124.45it/s]

100%|██████████| 1/1 [00:00<00:00, 125.50it/s]

100%|██████████| 1/1 [00:00<00:00, 124.22it/s]

100%|██████████| 1/1 [00:00<00:00, 123.14it/s]

100%|██████████| 1/1 [00:00<00:00, 119.80it/s]

100%|██████████| 1/1 [00:00<00:00, 124.73it/s]

100%|██████████| 1/1 [00:00<00:00, 125.14it/s]

100%|██████████| 1/1 [00:00<00:00, 124.85it/s]

100%|██████████| 1/1 [00:00<00:00, 124.9

100%|██████████| 1/1 [00:00<00:00, 125.25it/s]

100%|██████████| 1/1 [00:00<00:00, 124.15it/s]

100%|██████████| 1/1 [00:00<00:00, 124.62it/s]

100%|██████████| 1/1 [00:00<00:00, 125.20it/s]

100%|██████████| 1/1 [00:00<00:00, 125.07it/s]

100%|██████████| 1/1 [00:00<00:00, 126.31it/s]

100%|██████████| 1/1 [00:00<00:00, 125.41it/s]

100%|██████████| 1/1 [00:00<00:00, 125.26it/s]

100%|██████████| 1/1 [00:00<00:00, 124.04it/s]

100%|██████████| 1/1 [00:00<00:00, 125.91it/s]

100%|██████████| 1/1 [00:00<00:00, 124.98it/s]

100%|██████████| 1/1 [00:00<00:00, 124.08it/s]

100%|██████████| 1/1 [00:00<00:00, 123.73it/s]

100%|██████████| 1/1 [00:00<00:00, 124.58it/s]

  4%|▍         | 1000/26032 [20:15<8:24:36,  1.21s/it]Traceback (most recent call last):
  File "/nfs/homedirs/ayle/miniconda3/envs/gr/lib/python3.8/multiprocessing/queues.py", line 245, in _feed
    send_bytes(obj)
  File "/nfs/homedirs/ayle/miniconda3/envs/gr/lib/python3.8/multiprocessing/connection.py", line 200, in send_by

In [ ]:
# Attacks
from torchvision import datasets
from utils.constants import DATASET_PATH
mean = (0.4914, 0.4822, 0.4465)
std = (0.2471, 0.2435, 0.2616)
test_set = datasets.CIFAR10(root=DATASET_PATH, train=False, transform=transforms.ToTensor())
test_loader = torch.utils.data.DataLoader(
        test_set,
        batch_size=arguments['batch_size'],
        shuffle=False,
        pin_memory=True,
        num_workers=4
    )

trans = transforms.Normalize(mean, std)

# Attacks
ood_norms = []
attack_per_layer_norms = []

for i, (x, y) in enumerate(tqdm(test_loader)):
    
    model = deepcopy(backup_model)
    model.eval()
        
    adv_results, predictions = construct_adversarial_examples(x, y, 'FGSM', model, model.device, 8, False, False)
    _, advs, success = adv_results
    x = advs.cpu()
        
    new_x = []
    for image in x:
        image = trans(image.squeeze()).unsqueeze(0)
        new_x.append(image)
    x = torch.cat(new_x)
        
    model.eval()
    
    out = model(x.cuda())
    preds = out.argmax(dim=-1, keepdim=True).view_as(y)
    
    # get criterion
    criterion = find_right_model(
        CRITERION_DIR, arguments['prune_criterion'],
        model=model,
        limit=arguments['pruning_limit'],
        start=0.5,
        steps=arguments['snip_steps'],
        device=arguments['device'],
        arguments=arguments
    )
    
    criterion.prune(arguments['pruning_limit'],
                    train_loader=[(x, preds)],
                      ood_loader=None,
                      local=arguments['local_pruning'],
                      manager=None)
    
    layer_norms = []
    for j, (grad1, grad2) in enumerate(zip(orig_grads.values(), criterion.grads_abs.values())):
        if True:
#         if j not in [0, 7, 11, 20]:
            layer_norms.append(torch.norm(grad1 - grad2, p=2).cpu().numpy())
    attack_per_layer_norms.append(layer_norms)
    ood_norms.append(np.mean(layer_norms))

In [ ]:
# DS
class DS(Dataset):

    def __init__(self, images, labels):
        self.images = images
        self.labels = labels
        self.mean = [0.4914, 0.4822, 0.4465]
        self.std = [0.2471, 0.2435, 0.2616]
        self.transforms = transforms.Compose(
            [
                transforms.ToTensor(),
                transforms.Normalize(mean=self.mean, std=self.std)
            ]
        )

    def __getitem__(self, item):
        image = self.images[item] / 255
        image = self.transforms(image.transpose((1, 2, 0)))
        return image.to(torch.float32), torch.tensor(self.labels[item], dtype=torch.float32)

    def __len__(self):
        return len(self.images)

ds_path = os.path.join(DATASET_PATH, "cifar10_corrupted")
aurocs = []
ds_per_layer_norms = []
        
for ds_dataset_name in os.listdir(ds_path):
    if ds_dataset_name.endswith('5.npz'):
        _, test_loader = find_right_model(
            DATASETS, arguments['data_set'],
            arguments=arguments
        )
        npz_dataset = np.load(os.path.join(ds_path, ds_dataset_name))

        ds_dataset = DS(npz_dataset["images"], npz_dataset["labels"])
        ds_loader = torch.utils.data.DataLoader(
            ds_dataset,
            batch_size=arguments['batch_size'],
            shuffle=False,
            pin_memory=True,
            num_workers=4
        )

        ood_norms = []
        for i, (x, y) in enumerate(ds_loader):
            
            model = deepcopy(backup_model)
            model.eval()
            
            out = model(x.cuda())
            preds = out.argmax(dim=-1, keepdim=True).view_as(y)

            # get criterion
            criterion = find_right_model(
                CRITERION_DIR, arguments['prune_criterion'],
                model=model,
                limit=arguments['pruning_limit'],
                start=0.5,
                steps=arguments['snip_steps'],
                device=arguments['device'],
                arguments=arguments
            )

            criterion.prune(arguments['pruning_limit'],
                              train_loader=[(x, preds)],
                              ood_loader=None,
                              local=arguments['local_pruning'],
                              manager=None)

            layer_norms = []
            for grad1, grad2 in zip(orig_grads.values(), criterion.grads_abs.values()):
                layer_norms.append(torch.norm(grad1 - grad2, p=2).cpu().numpy())
            ds_per_layer_norms.append(layer_norms)
            ood_norms.append(np.mean(layer_norms))

        auroc = calculate_auroc(np.concatenate((np.zeros_like(norms), np.ones_like(ood_norms))), np.concatenate((norms, ood_norms)))
        print('AUROC', auroc)
        aurocs.append(auroc)
        
print('Mean AUROC', np.mean(aurocs))

In [23]:
np.mean(ood_norms)

4990.221

In [24]:
norms = np.array(norms)
ood_norms = np.array(ood_norms)

In [25]:
calculate_auroc(np.concatenate((np.zeros_like(norms), np.ones_like(ood_norms))), np.concatenate((norms, ood_norms)))

0.853601

In [ ]:
# 0.9999251152073733 batch size 10 dropout eval model train, not layers 0 3 7
# 0.9999873271889401 batch size 10 model train, not layers 0 3 7
# 0.9999592933947773 batch size 10 model train, all layers

In [ ]:
plt.plot(np.array(per_layer_norms).mean(0), label='In-distribution')
plt.plot(np.array(ood_per_layer_norms).mean(0), label='Out-Of-Distribution')
# plt.plot(np.array(attack_per_layer_norms).mean(0), label='Attack (FGSM \u03B5 = 8)')
# plt.plot(np.array(ds_per_layer_norms).mean(0), label='Distribution Shifts')
plt.legend()
plt.xlabel('ResNet18 layers')
plt.ylabel('L2 distance to original scores')

In [ ]:
# np.array(ood_per_layer_norms).mean(0)

In [ ]:
thresholds, _ = torch.topk(torch.tensor(norms), int(len(norms)*0.05), sorted=True)

In [ ]:
threshold = thresholds[-1]

In [ ]:
num_detected = 0
for ood_norm in ood_norms:
    if ood_norm >= threshold:
        num_detected += 1

In [ ]:
num_detected / len(ood_norms)